# lil_cluster
Фундаментальная модель в ансамбле

Изучив специфику авторынка стало понятно, что бренд и модель - названия, не имеюще общих правил. Поэтому модель их может использовать только в качетсве dummy переменных. Но 500 с лишним переменных для меня показались слишком большим количеством для 80 тысяч автомобилей. Поэтому я решил разделить их специфику по моделям. Тут становится интереснее. Оказывается автомобили одной и той же модели, но разных комплектаций могут кратно отличаться в цене. Исходя из этого инсайта я понял, что можно сравнивать автомобили только в рамках одной модели. Этим и будет заниматься lil_cluster

Оказалось, что в датасете множество машин одной и той же модели, более того, в тренировочном датасете оказалось 98% наименований моделей, покрывающие наименования тестового датасета. Исходя из этого, lil_cluster будет иметь главный решающий голос в ансамбле в случаях, когда модель машины ему хорошо известна

In [91]:
%store -r df

Для этой модели потребуется сократить разновидности моделей в тренировочной выборки до соответствия их тестовой

In [169]:
test = df[df.price==-1]
train = df[df.price!=-1]

In [93]:
print(f'''{test.model_name.nunique()} - количество уникальных моделей 
{len(test.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - \
уникальны для всех предоставленных брендов''')

544 - количество уникальных моделей 
544 - количествоуникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - уникальны для всех предоставленных брендов


In [94]:
print(f'''{train.model_name.nunique()} - количество уникальных моделей 
{len(train.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название''')

1086 - количество уникальных моделей 
1105 - количествоуникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название


In [227]:
test = df[df.price==-1]
train = df[df.price!=-1]

### Исключает модели, которых нет в  ###
filtered_train = train[train.model_name.isin(test.model_name.unique())
                      ].reset_index(drop=True)



'''Делает неуникальные для разных брендов названия моделей - уникальными'''
###############################################################################
grouped_ft = filtered_train.groupby(['brand','model_name']).count()

valco_non_unique = grouped_ft.reset_index(level=1).model_name.value_counts()[
    grouped_ft.reset_index(level=1).model_name.value_counts()!=1]

def change_name_to_unique(df):
    df_non_unique = df[
        df.model_name.isin(valco_non_unique.index)]

    for i in df_non_unique.index:
        df.at[i,'model_name'] = \
        df.iloc[i].brand + '_' + df.iloc[i].model_name

change_name_to_unique(filtered_train)
change_name_to_unique(test)

### Удаляет модели в трейне, которых нет в тесте после уточнения модели брендом
models_to_drop = (set(filtered_train.model_name.unique()) -set(test.model_name.unique()))
index_to_drop = filtered_train[filtered_train.model_name.isin(models_to_drop)].index

filtered_train.drop(index_to_drop,inplace=True)
###############################################################################


In [232]:
display(set(test.model_name.unique())- set(filtered_train.model_name.unique()))
print('18 Моделей из тестового датасета, что нет в тренировочном')

{'300ZX',
 'ARTEON',
 'COROLLA_II',
 'GOLF_R32',
 'IST',
 'LANCER_RALLIART',
 'LM',
 'OTTI',
 'POLO_GTI',
 'PORTE',
 'PRESEA',
 'RSQ3',
 'SPARKY',
 'TORNEO',
 'WILL_CYPHA',
 'X4_M',
 'Z',
 'ZEST'}

18 Моделей из тестового датасета, что нет в тренировочном
